<h2>
Pairs Trading
</h2>
<blockquote>
<p>
Pairs trading is an approach that takes advantage of the
mispricing between two (or more) co-moving assets, by
taking a long position in one(many) and shorting the
other(s), betting that the relationship will hold and that
prices will converge back to an equilibrium level.
</p>
<p>
<i>Definitive Guide to Pairs Trading</i> availabel from <a href="https://hudsonthames.org/">Hudson and Thames</a>
</p>
</blockquote>
<p>
Pairs trading is a so called statistical arbitrage trading strategy.
</p>
<blockquote>
<p>
Statistical arbitrage and pairs trading tries to solve this problem using price relativity. If two assets share the same
characteristics and risk exposures, then we can assume that their behavior would be similar as well. This has
the benefit of not having to estimate the intrinsic value of an asset but rather just if it is under or overvalued
relative to a peer(s). We only have to focus on the relationship between the two, and if the spread happens
to widen, it could be that one of the securities is overpriced, the other is underpriced, or the mispricing is a
combination of both.
</p>
<p>
<i>Definitive Guide to Pairs Trading</i> availabel from <a href="https://hudsonthames.org/">Hudson and Thames</a>
</p>
</blockquote>
<p>
Pairs trading algorithms have been reported to yield portfolios with Sharpe ratios in excess of 1.0 and returns of 10% or
higher. Pairs trading takes both long and short positions, so the portfolio tends to be market neutral. A pairs trading portfolio
can have drawdowns, but the drawdowns are much less than a benchmark like the S&P 500.
</p>
<p>
Markets tend toward efficiency and many quantitative approaches fade over time as they are adopted by hedge funds. Pairs trading
goes back to the mid-1980s. The approach still seems to be profitable. The reason for this could be that there are a vast
number of possible pairs and the pairs portfolio's tend to be fairly small (5 to 20 pairs, in most cases). This may always
leave unexploited pairs in the market.
</p>
<p>
This Python notebook investigates pairs trading algorithms in an attempt reproduce the reported results.
</p>
<h3>
Overview
</h3>
<p>
Pairs trading algorithms attempt to identify pairs of stocks that have mean reverting behavior. Mean reversion takes place
around a common mean between the two stocks. The strategy is based on the idea that when one of the pairs is above or below
the mean, it will tend to revert back to the mean.
</p>
<p>
The stock that is above the mean is shorted, on the speculation that it's price will decrease as it reverts toward the mean.
The stock below the mean is bought, on the speculation that it's will increase as it reverts toward the mean.
</p>
<p>
Pairs trading is a "market neutral" strategy since both long a short positions are taken in related stocks.
</p>
<p>
Pairs trading involves two logical steps:
</p>
<ol>
<li>
<p>
Identify a pair of stocks that are likely to have mean reverting behavior using a lookback period.
</p>
</li>
<li>
<p>
Trading the stocks using the long/short strategy over the trading period.
</p>
</li>
</ol>
<h3>
Pairs Identification
</h3>
<p>
In their paper, the authors Quynh Bui and Robert Ślepaczuk write that they found correlation (on the stock prices) to be a
stronger identification of pairs than either cointegration or the
<a href="http://bearcave.com/misl/misl_tech/wavelets/hurst/index.html">Hurst exponent</a>.
</p>
<p>
The problem with using correlation alone is that stocks with high correlation may do not necessarly have mean reversion
behavior. A better approach is to use correlation as the first filter and then apply cointegration to determine whether
the stocks are mean reverting.
</p>
<h3>
Cointegration
</h3>
<p>
Cointegration is used to recognize asset pairs that are canidates for pairs trading. See the reference section for discussions on cointegration.
</p>
<p>
There are two common cointegration tests:
</p>
<ul>
<li>
Engle-Granger (linear regression apporach).
</li>
<li>
Johansen
</li>
</ul>
<p>
A number of authors perfer the Johansen test. Kris Longmore (of Robot Wealth, see references)
prefers to Engle-Granger linear regression. Engle-Granger is easier to understand and is used in this notebook.
</p>
<h3>
Finding Paris in Stock Market Sectors
</h3>
<p>
In their paper on pairs trading, Quynh Bui and Robert Ślepaczuk select pairs from the NASDAQ 100 index. From the index they form over 5,000
pairs and evaluate each pair as a canidate for pairs trading.
</p>
<p>
This notebook focuses on stocks from the S&P 500 index. These stocks have a high daily trading volume, low spread and are easy to borrow
if the stock is selected for shorting.
</p>
<p>
There are 124,750 unique pairs composed of S&P 500 stocks.  This is far to large a universe of stocks to exhasutively search.
To reduce the search space, pairs are formed from the eleven S&P 500 sectors. Stocks in a particular sector should have similar
behavior and so would be good candidates for pairs.
</p>
<p>
The stocks, in each sector were downloaded from barchart.com and filtered. The sectors are:
</p>
<ol>
<li>
Consumer discressionary
</li>
<li>
Consumer staples
</li>
<li>
Energy
</li>
<li>
Financials
</li>
<li>
Health care
</li>
<li>
Industrials
</li>
<li>
Info tech
</li>
<li>
Materials
</li>
<li>
Real estate
</li>
<li>
Communication services
</li>
<li>
Utilities
</li>
</ol>



<h2>
References
</h2>
<ol>
<li>
<a href="https://www.researchgate.net/publication/5217081_Pairs_Trading_Performance_of_a_Relative_Value_Arbitrage_Rule">Pairs Trading: Performance of a Relative Value Arbitrage Rule</a>, February 2006
</li>
<li>
<a href="https://www.quantconnect.com/tutorials/strategy-library/intraday-dynamic-pairs-trading-using-correlation-and-cointegration-approach">Intraday Dynamic Pairs Trading using Correlation and Cointegration</a>
</li>
<li>
<a href="https://bsic.it/pairs-trading-building-a-backtesting-environment-with-python/">Pairs Trading: building a backtesting environment with Python</a>
</li>
<li>
<a href="https://www.sciencedirect.com/science/article/pii/S037843712100964X">Applying Hurst Exponent in pair trading strategies
on Nasdaq 100 index</a>
by Quynh Bui and Robert Ślepaczuk
</li>
<li>
<a href="https://www.sciencedirect.com/science/article/pii/S2214845021000880">Pairs trading: is it applicable to exchange-traded funds?</a>
</li>
<li>
<a href="https://hudsonthames.org/an-introduction-to-cointegration/">An Introduction to Cointegration for Pairs Trading By Yefeng Wang</a>
</li>
<li>
<a href="https://www.tradelikeamachine.com/blog/cointegration-pairs-trading/part-1-using-cointegration-for-a-pairs-trading-strategy"><i>Using Cointegration for a Pairs Trading Strategy</i> Martyn Tinsley</a>
</li>
<li>
<a href="https://letianzj.github.io/cointegration-pairs-trading.html">Quantitative Trading and Systematic Investing by Letian Wang</a> This
post includes a discussion on how the results of Johansen cointegration can be interpreted.
</li>
<li>
<a href="https://robotwealth.com/practical-pairs-trading/">Pairs Trading on the Robot Wealth blog by Kris Longmore</a>
</li>
<li>
<a href="https://pykalman.github.io/">pykalman: the dead-simple Kalman Filter, Kalman Smoother, and EM library for Python</a>
</li>
</ol>